In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

In [3]:
DATASET_SIZE = 480
DATASET_IS_BALANCED = True


MAX_FEATURES = 20000        # max_features params for CountVectorizer

training_args_datetime = datetime(year=2024, month=2, day=26)
training_name = 'glove-cnn-{}_{}k_{}_{}'.format(
    MAX_FEATURES,
    DATASET_SIZE,
    'bal' if DATASET_IS_BALANCED else 'imbal',
    training_args_datetime.strftime('%Y-%m-%d')
)

training_storing_folder = Path(training_name).resolve()

In [5]:
import pickle

import tensorflow as tf
import keras

# load the tf model
# either a end-to-end
# or build our own (by loading the vectorizer and the model)


text_vectorizer_path = Path.joinpath(training_storing_folder, "{}_textvectorizer.pkl".format(
    training_name,
    # training_args_datetime.strftime("%Y-%m-%d")
))
vectorizer_from_disk = pickle.load(open(text_vectorizer_path, 'rb'))
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=512)

vectorizer.set_weights(vectorizer_from_disk['weights'])

model_path = Path.joinpath(training_storing_folder, "{}_model.keras".format(
    training_name,
    # training_args_datetime.strftime("%Y-%m-%d")
))
model = keras.models.load_model(model_path)

end_to_end_model_path = Path.joinpath(training_storing_folder, "{}_end2end.keras".format(
    training_name,
    # training_args_datetime.strftime("%Y-%m-%d")
))

end_to_end_model = keras.models.load_model(end_to_end_model_path)

print('\n\n')
print('Loaded text vectorizer from {}'.format(text_vectorizer_path))
print('Loaded model from {}'.format(model_path))
print('Loaded end to end model from {}'.format(end_to_end_model_path))

2024-03-15 11:20:31.851189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:31.914759: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:31.914798: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:31.916219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:31.916247: I external/local_xla/xla/stream_executor




Loaded text vectorizer from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_bal_2024-02-26/glove-cnn-20000_480k_bal_2024-02-26_textvectorizer.pkl
Loaded model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_bal_2024-02-26/glove-cnn-20000_480k_bal_2024-02-26_model.keras
Loaded end to end model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_bal_2024-02-26/glove-cnn-20000_480k_bal_2024-02-26_end2end.keras


In [6]:
end_to_end_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 512)               0         
 ctorization)                                                    
                                                                 
 model (Functional)          (None, 2)                 6808154   
                                                                 
Total params: 6808154 (25.97 MB)
Trainable params: 6808154 (25.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
vectorizer.get_vocabulary()[:20]

['',
 '[UNK]',
 'the',
 'and',
 'to',
 'a',
 'game',
 'i',
 'it',
 'of',
 'you',
 'is',
 'this',
 'in',
 'that',
 'for',
 'but',
 's',
 'with',
 't']

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 512, 300)             6000600   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                              

In [9]:
# save the .keras model to SavedModel format
# which include a .pb file

# https://www.tensorflow.org/tutorials/keras/save_and_load#savedmodel_format

# model.save(Path.joinpath(training_storing_folder, "{}_{}_model_savedmodel".format(
#     training_name,
#     training_args_datetime.strftime("%Y-%m-%d")
# )))

In [10]:
# save the pretrained model to onnx

import tf2onnx
import onnx

onnx_model_path = Path.joinpath(training_storing_folder, "{}_modelonly.onnx".format(
    training_name,
    # training_args_datetime.strftime("%Y-%m-%d")
))

print(training_storing_folder)
print('\n\n')
print(onnx_model_path)

# Keras models and tf functions and can be converted directly within python
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=[tf.TensorSpec([None,512], dtype=tf.int32, name='input')],     # match the input name of the model
    # extra_opset=['ai.onnx.contrib:1'],
    opset=15        # default opset of tf2onnx for testing
)

onnx.save(onnx_model, onnx_model_path)

Could not search for non-variable resources. Concrete function internal representation may have changed.


/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_bal_2024-02-26



/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_bal_2024-02-26/glove-cnn-20000_480k_bal_2024-02-26_2024-02-26_modelonly.onnx
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


2024-03-15 11:20:51.212133: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:51.212182: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-03-15 11:20:51.212250: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-03-15 11:20:51.212450: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:51.212468: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 11:20:51.212478: I external/local_xla/xl

In [11]:
# https://github.com/onnx/tensorflow-onnx/issues/1867
# "probably there's no shared-name keyword for hash tables in TextVectorization layer."
# Therefore we only convert the model without the vectorizer (resulting in like huggingface, that the tokenizer is not included in onnx)

# !python -m tf2onnx.convert --saved-model "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model_savedmodel" --output "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_modelonly.onnx" --extra_opset ai.onnx.contrib:1 --opset 15

In [12]:
import onnxruntime as rt

sess = rt.InferenceSession(
    onnx_model_path,
    providers=['CPUExecutionProvider']
)

input_name = [inp.name for inp in sess.get_inputs()][0]     # only one input in this model
label_names = [label.name for label in sess.get_outputs()]  # it outputs the label and the probability

In [13]:
test_data = [['I like the game'], ["I do not like it."], ["It crashes when I just run on my pc."]]

In [14]:
# test inference

pred_keras = []
perd_onnx = []

for i in range(len(test_data)):
    # keras inference
    pred_keras.append(end_to_end_model.predict(test_data[i]))

    # onnx inference
    v_out = vectorizer(test_data[i])
    perd_onnx.append(sess.run(None, {"input": v_out.cpu().numpy().astype(np.int32)}))

print(pred_keras)
print(perd_onnx)

for i in range(len(test_data)):
    result_keras = pred_keras[i]
    result_onnx = perd_onnx[i][0]

    np.testing.assert_allclose(result_keras, result_onnx, rtol=1e-3, atol=1e-3)

2024-03-15 11:21:01.001638: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900


1/1 [==============================] - 1s 913ms/step
Instructions for updating:
Use tf.identity with explicit device placement instead.
1/1 [==============================] - 0s 18ms/step
[array([[0.24413665, 0.75586337]], dtype=float32), array([[0.8491382 , 0.15086186]], dtype=float32), array([[0.8683612 , 0.13163884]], dtype=float32)]
[[array([[0.24412242, 0.7558776 ]], dtype=float32)], [array([[0.84912586, 0.15087417]], dtype=float32)], [array([[0.8683908 , 0.13160917]], dtype=float32)]]
